In [1]:
import islpy as isl
from src.io_tools import *
from src.graph_tools import *
from src.swap_tools import *
from prettytable import PrettyTable

In [2]:
import warnings
import logging

warnings.filterwarnings("ignore")
logging.getLogger('stevedore.extension').setLevel(logging.CRITICAL)


In [3]:
json_file_path = 'benchmarks/polyhedral/queko-bss-16qbt/16QBT_100CYC_QSE_9.json'

data = json_file_to_isl(json_file_path)

In [4]:

domain, access, schedule = filter_multi_qubit_gates(data["domain"], data["read_dependencies"], data["schedule"])


In [5]:
regex = re.compile(r'(?<=\[)(.*?)(?=\])')
num_qubits = int(regex.findall(data['read_dependencies'].range().as_set().lexmax().to_str())[0]) + 1

In [6]:
physical_qubits_domain = isl.Set(f"{{ [i] : 1 <= i <= {num_qubits} }}")

In [7]:
# Generate a 2D grid for 200 qubits (10 rows and 20 columns)
def generate_2d_grid(num_rows, num_cols):
    num_qubits = num_rows * num_cols
    graph = [[] for _ in range(num_qubits)]
    
    for i in range(num_rows):
        for j in range(num_cols):
            index = i * num_cols + j
            # Connect to the left neighbor
            if j > 0:
                graph[index].append(index - 1)
            # Connect to the right neighbor
            if j + 1 < num_cols:
                graph[index].append(index + 1)
            # Connect to the top neighbor
            if i > 0:
                graph[index].append(index - num_cols)
            # Connect to the bottom neighbor
            if i + 1 < num_rows:
                graph[index].append(index + num_cols)
    
    return graph

num_rows = 4
num_cols = 4
graph = generate_2d_grid(num_rows, num_cols)



In [8]:
backend_disconnected_edges =  extract_edges_map(graph)
shortest_paths = extract_shortest_paths(graph)

In [9]:
initial_mapping = isl.Map(f"{{ q[i] -> [i] : 1<=i<={num_qubits} }}")

In [10]:
mapping = initial_mapping
swap_count = 0
programme = schedule


In [11]:
import time

In [12]:
programme

Map("{ [i0] -> q[11279 - 11i0] : 1024 <= i0 <= 1025; [i0] -> q[8261 - 10i0] : 825 <= i0 <= 826; [i0] -> q[5413 - 9i0] : 600 <= i0 <= 601; [i0] -> q[4253 - 8i0] : 530 <= i0 <= 531; [i0] -> q[4165 - 6i0] : 693 <= i0 <= 694; [i0] -> q[3908 - 4i0] : 976 <= i0 <= 977; [i0] -> q[3232 - 3i0] : 1075 <= i0 <= 1076; [i0] -> q[2890 - 3i0] : 962 <= i0 <= 963; [i0] -> q[2816 - 3i0] : 936 <= i0 <= 937; [i0] -> q[2721 - 3i0] : 906 <= i0 <= 907; [i0] -> q[2461 - 8i0] : 306 <= i0 <= 307; [i0] -> q[2103 - 2i0] : 1044 <= i0 <= 1045; [i0] -> q[1624 - 8i0] : 202 <= i0 <= 203; [i0] -> q[1605 - 3i0] : 530 <= i0 <= 531; [i0] -> q[1538 - 2i0] : 768 <= i0 <= 769; [i0] -> q[1262 - 7i0] : 179 <= i0 <= 180; [i0] -> q[1214 - 2i0] : 600 <= i0 <= 601; [i0] -> q[1080 - 6i0] : 179 <= i0 <= 180; [i0] -> q[1049 - 3i0] : 345 <= i0 <= 346; [i0] -> q[933 - 3i0] : 306 <= i0 <= 307; [i0] -> q[434 - i0] : 421 <= i0 <= 422; [i0] -> q[287 - 8i0] : 34 <= i0 <= 35; [i0] -> q[159 - i0] : 144 <= i0 <= 145; [i0] -> q[-67 + 2i0] : 34 

In [13]:
iteration = 0
start = time.time()
while not programme.is_empty():
    #print(f"Iteration {iteration}")
    #print(programme.domain())
    iteration += 1
    programme_mapped = programme.apply_range(mapping)
    first_programme_mapped = programme_mapped.lexmin()
    second_programme_mapped = programme_mapped.subtract(first_programme_mapped)
    programme_access = first_programme_mapped.flat_range_product(second_programme_mapped)
    first_disconnected_edge = programme_access.intersect_range(backend_disconnected_edges).domain().lexmin()
    if first_disconnected_edge.is_empty():
        break

    disconected_qubits = programme_mapped.intersect_domain(first_disconnected_edge).range().as_set()
    q1 , q2 = disconected_qubits.dim_min_val(0).to_python(),disconected_qubits.dim_max_val(0).to_python()
    swap_count += shortest_paths[q1]['costs'][q2] -1
    new_domain = programme.domain().as_set().lex_gt_set(first_disconnected_edge.as_set()).domain()
    programme = programme.intersect_domain(new_domain).coalesce()
    swap_map = swaps_to_isl_map(shortest_paths[q1]['paths'][q2])
    mapping = apply_swaps_to_logical_qubits_map(swap_map,mapping,physical_qubits_domain)
    

end = time.time()


In [14]:
swap_count

151

In [15]:
iteration

110

In [16]:
results_table = PrettyTable()

results_table.field_names = ["Method","Swap Count", "Execution Time (s)"]
results_table.add_row(["Our solution", swap_count, end-start])

----------------------

In [17]:
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import CouplingMap, PassManager ,Layout
from qiskit.transpiler.passes import LookaheadSwap, SabreSwap
from qiskit_aer import Aer

In [18]:
qasm_code = json_file_qasm(json_file_path)
qc = QuantumCircuit.from_qasm_str(qasm_code)

num_qubits = qc.num_qubits

In [19]:

#qc.draw(output='mpl')

In [20]:
edges = [(i, j) for i, neighbors in enumerate(graph) for j in neighbors if i < j]


coupling_map = CouplingMap(edges)

In [21]:
layout_dict = {qc.qubits[i]: i for i in range(num_qubits)}  
layout = Layout(layout_dict)

In [22]:
pass_manager = PassManager()
pass_manager.append(LookaheadSwap(coupling_map))

In [23]:
qc.count_ops()

OrderedDict([('x', 816), ('cx', 320)])

In [24]:
start_time = time.time()
transpiled_circuit = transpile(qc, coupling_map=coupling_map, initial_layout=layout)
optimized_circuit = pass_manager.run(transpiled_circuit)
end_time = time.time()

In [25]:
swap_count = sum(1 for op in transpiled_circuit.data if op[0].name == 'swap')

In [26]:
results_table.add_row(["LookaheadSwap", swap_count, end_time-start_time])

In [27]:
pass_manager = PassManager()
pass_manager.append(SabreSwap(coupling_map))

start_time = time.time()
transpiled_circuit = transpile(qc, coupling_map=coupling_map, initial_layout=layout)
optimized_circuit = pass_manager.run(transpiled_circuit)
end_time = time.time()

swap_count = sum(1 for op in transpiled_circuit.data if op[0].name == 'swap')

results_table.add_row(["SabreSwap", swap_count, end_time-start_time])

In [28]:
start_time = time.time()
transpiled_circuit = transpile(qc, coupling_map=coupling_map, initial_layout=layout,optimization_level=0)
end_time = time.time()
swap_count = sum(1 for op in transpiled_circuit.data if op[0].name == 'swap')

results_table.add_row(["without optimizer", swap_count, end_time-start_time])

In [29]:
print(results_table)

+-------------------+------------+---------------------+
|       Method      | Swap Count |  Execution Time (s) |
+-------------------+------------+---------------------+
|    Our solution   |    151     |   38.8696825504303  |
|   LookaheadSwap   |     30     |  0.5446712970733643 |
|     SabreSwap     |     22     |  0.1252748966217041 |
| without optimizer |    128     | 0.07019400596618652 |
+-------------------+------------+---------------------+
